# XGBoost快速入门——与scikit-learn一起使用-GridSearchcv

In [1]:
# 运行 xgboost安装包中的示例程序
from xgboost import XGBClassifier

# 加载LibSVM格式数据模块
from sklearn.datasets import load_svmlight_file

from sklearn.grid_search import GridSearchCV

from sklearn.metrics import accuracy_score

from matplotlib import pyplot

/Applications/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Applications/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## 数据读取

scikit-learn支持多种格式的数据，包括LibSVM格式数据
XGBoost可以加载libsvm格式的文本数据，libsvm的文件格式（稀疏特征）如下：
1 101:1.2 102:0.03
0 1:2.1 10001:300 10002:400
...

每一行表示一个样本，第一行的开头的“1”是样本的标签。“101”和“102”为特征索引，'1.2'和'0.03' 为特征的值。
在两类分类中，用“1”表示正样本，用“0” 表示负样本。也支持[0,1]表示概率用来做标签，表示为正样本的概率。

下面的示例数据需要我们通过一些蘑菇的若干属性判断这个品种是否有毒。
UCI数据描述：http://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/ ，
每个样本描述了蘑菇的22个属性，比如形状、气味等等(加工成libsvm格式后变成了126维特征)，
然后给出了这个蘑菇是否可食用。其中6513个样本做训练，1611个样本做测试。

XGBoost加载的数据存储在对象DMatrix中
XGBoost自定义了一个数据矩阵类DMatrix，优化了存储和运算速度
DMatrix文档：http://xgboost.readthedocs.io/en/latest/python/python_api.html

In [2]:
# read in data，数据在xgboost安装的路径下的demo目录,现在copy到代码目录下的data目录
my_workpath = './data/'
X_train,y_train = load_svmlight_file(my_workpath + 'agaricus.txt.train')
X_test,y_test = load_svmlight_file(my_workpath + 'agaricus.txt.test')

## 设置模型训练参数
暂时没用

In [3]:
# specify parameters via map
params = {'max_depth':2, 'eta':0.1, 'silent':0, 'objective':'binary:logistic' }
print params

{'objective': 'binary:logistic', 'eta': 0.1, 'max_depth': 2, 'silent': 0}


## 构造模型

In [4]:
#bst = XGBClassifier(param)
bst =XGBClassifier(max_depth=2, learning_rate=0.1, silent=True, objective='binary:logistic')

## 交叉验证
会比较慢:(

In [7]:
# 设置boosting迭代计算次数
param_test = {
 'n_estimators': range(1, 51, 1)
}
clf = GridSearchCV(estimator = bst, param_grid = param_test, scoring='accuracy', cv=5)
clf.fit(X_train, y_train)
clf.grid_scores_, clf.best_params_, clf.best_score_

([mean: 0.90542, std: 0.08996, params: {'n_estimators': 1},
  mean: 0.94749, std: 0.07458, params: {'n_estimators': 2},
  mean: 0.90542, std: 0.08996, params: {'n_estimators': 3},
  mean: 0.94749, std: 0.07458, params: {'n_estimators': 4},
  mean: 0.90573, std: 0.09028, params: {'n_estimators': 5},
  mean: 0.94718, std: 0.07436, params: {'n_estimators': 6},
  mean: 0.90542, std: 0.08996, params: {'n_estimators': 7},
  mean: 0.94242, std: 0.07331, params: {'n_estimators': 8},
  mean: 0.94473, std: 0.07739, params: {'n_estimators': 9},
  mean: 0.97482, std: 0.02235, params: {'n_estimators': 10},
  mean: 0.94887, std: 0.07621, params: {'n_estimators': 11},
  mean: 0.97850, std: 0.02387, params: {'n_estimators': 12},
  mean: 0.97298, std: 0.03186, params: {'n_estimators': 13},
  mean: 0.97850, std: 0.02387, params: {'n_estimators': 14},
  mean: 0.97298, std: 0.03186, params: {'n_estimators': 15},
  mean: 0.97850, std: 0.02387, params: {'n_estimators': 16},
  mean: 0.97850, std: 0.02387, pa

# 测试

In [8]:
#make prediction
preds = clf.predict(X_test)
predictions = [round(value) for value in preds]

test_accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy of gridsearchcv: %.2f%%" % (test_accuracy * 100.0))

Test Accuracy of gridsearchcv: 97.27%
